## Basics of Flask:

### 1. Create a Flask app that displays "Hello, World!" on the homepage.

In [ ]:
from flask import Flask

app = Flask(__name__)

@app.route('/')
def hello_world():
    return 'Hello, World!'

if __name__ == '__main__':
    app.run(debug=True)

### 2. Build a Flask app with static HTML pages and navigate between them.

In [ ]:
from flask import Flask, render_template

app = Flask(__name__)

@app.route('/')
def home():
    return render_template('home.html')

@app.route('/')
def about():
    return render_template('about.html')

if __name__ == '__main__':
    app.run(debug=True)

In [ ]:
static html pages

home.html:

about.html:

In [ ]:
<!DOCTYPE html>
<html>
<head>
    <title>About</title>
    <link rel="stylesheet" href="{{ url_for('static', filename='style.css') }}">
</head>
<body>
    <h1>About Us</h1>
    <p>This is the about page content.</p>
    <a href="{{ url_for('home') }}">Go to Home Page</a>
</body>
</html>

### 3. Develop a Flask app that uses URL parameters to display dynamic content.

In [ ]:
from flask import Flask, render_template

app = Flask(__name__)

# Route that takes a dynamic parameter in the URL
@app.route('/user/<username>')
def show_user_profile(username):
    return f'Hello, {username}!'

# Route that takes multiple dynamic parameters in the URL
@app.route('/post/<int:post_id>')
def show_post(post_id):
    return f'Post ID: {post_id}'

if __name__ == '__main__':
    app.run(debug=True)

### 4. Create a Flask app with a form that accepts user input and displays it.

In [ ]:
from flask import Flask, render_template, request, redirect, url_for

app = Flask(__name__)

# Default route with a form
@app.route('/', methods=['GET', 'POST'])
def index():
    if request.method == 'POST':
        # Accessing the form data submitted by the user
        user_input = request.form['user_input']
        # Redirecting to another route to display the submitted data
        return redirect(url_for('display', input_data=user_input))
    return render_template('index.html')

# Route to display the submitted data
@app.route('/display/<input_data>')
def display(input_data):
    return render_template('display.html', data=input_data)

if __name__ == '__main__':
    app.run(debug=True)

### 5. Implement user sessions in a Flask app to store and display user-specific data.

In [ ]:
from flask import Flask, render_template, request, redirect, url_for, session

app = Flask(__name__)
# Set a secret key for sessions (you should ideally use a secret random value)
app.secret_key = 'your_secret_key_here'

# Route to set user-specific data in the session
@app.route('/set_data', methods=['POST'])
def set_data():
    if request.method == 'POST':
        # Get user data from the form
        user_data = request.form['user_data']
        # Store user data in the session
        session['user_data'] = user_data
        return redirect(url_for('get_data'))
    return render_template('set_data.html')

# Route to retrieve and display user-specific data from the session
@app.route('/get_data')
def get_data():
    user_data = session.get('user_data')  # Retrieve user data from the session
    return render_template('get_data.html', user_data=user_data)

if __name__ == '__main__':
    app.run(debug=True)


## Intermediate Flask Topics:

### 6. Build a Flask app that allows users to upload files and display them on the website.

In [ ]:
import os
from flask import Flask, render_template, request, redirect, url_for

app = Flask(__name__)
UPLOAD_FOLDER = 'uploads'
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER

# Route to render the file upload form
@app.route('/')
def index():
    return render_template('index.html')

# Route to handle file upload and display uploaded files
@app.route('/upload', methods=['POST'])
def upload_file():
    if request.method == 'POST':
        # Check if the POST request has a file part
        if 'file' not in request.files:
            return redirect(request.url)

        file = request.files['file']
        # If the user does not select a file, the browser submits an empty file without a filename
        if file.filename == '':
            return redirect(request.url)

        if file:
            # Save the uploaded file to the UPLOAD_FOLDER
            file.save(os.path.join(app.config['UPLOAD_FOLDER'], file.filename))
            return redirect(url_for('uploaded_file', filename=file.filename))

    return render_template('index.html')

# Route to display the uploaded file
@app.route('/uploads/<filename>')
def uploaded_file(filename):
    return render_template('uploaded_file.html', filename=filename)

if __name__ == '__main__':
    # Ensure the 'uploads' directory exists before running the app
    if not os.path.exists(UPLOAD_FOLDER):
        os.makedirs(UPLOAD_FOLDER)
    app.run(debug=True)


### 7. Integrate a SQLite database with Flask to perform CRUD operations on a list of items.

In [ ]:
from flask import Flask, render_template, request, redirect, url_for
from flask_sqlalchemy import SQLAlchemy

app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///items.db'
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
db = SQLAlchemy(app)

# Define a model for the items
class Item(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    name = db.Column(db.String(100), nullable=False)

    def __repr__(self):
        return f'<Item {self.id}>'

# Create the SQLite database
db.create_all()

@app.route('/')
def index():
    # Fetch all items from the database
    items = Item.query.all()
    return render_template('index.html', items=items)

# Route to add new items
@app.route('/add', methods=['POST'])
def add_item():
    if request.method == 'POST':
        name = request.form['name']
        new_item = Item(name=name)
        db.session.add(new_item)
        db.session.commit()
    return redirect(url_for('index'))

# Route to delete items
@app.route('/delete/<int:item_id>')
def delete_item(item_id):
    item = Item.query.get_or_404(item_id)
    db.session.delete(item)
    db.session.commit()
    return redirect(url_for('index'))

if __name__ == '__main__':
    app.run(debug=True)

### 8. Implement user authentication and registration in a Flask app using Flask-Login.

In [ ]:
from flask import Flask, render_template, request, redirect, url_for
from flask_login import LoginManager, UserMixin, login_user, login_required, logout_user, current_user
from flask_sqlalchemy import SQLAlchemy
from werkzeug.security import generate_password_hash, check_password_hash

app = Flask(__name__)
app.secret_key = 'your_secret_key_here'
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///users.db'
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
db = SQLAlchemy(app)
login_manager = LoginManager()
login_manager.init_app(app)

# Define the User model for the database
class User(UserMixin, db.Model):
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(100), unique=True, nullable=False)
    password = db.Column(db.String(100), nullable=False)

@login_manager.user_loader
def load_user(user_id):
    return User.query.get(int(user_id))

# Create the SQLite database
db.create_all()

@app.route('/')
@login_required
def home():
    return render_template('home.html')

@app.route('/login', methods=['GET', 'POST'])
def login():
    if request.method == 'POST':
        username = request.form['username']
        password = request.form['password']
        user = User.query.filter_by(username=username).first()
        if user and check_password_hash(user.password, password):
            login_user(user)
            return redirect(url_for('home'))
    return render_template('login.html')

@app.route('/register', methods=['GET', 'POST'])
def register():
    if request.method == 'POST':
        username = request.form['username']
        password = request.form['password']
        hashed_password = generate_password_hash(password)
        new_user = User(username=username, password=hashed_password)
        db.session.add(new_user)
        db.session.commit()
        return redirect(url_for('login'))
    return render_template('register.html')

@app.route('/logout')
@login_required
def logout():
    logout_user()
    return redirect(url_for('login'))

if __name__ == '__main__':
    app.run(debug=True)


In [ ]:
home.html:

In [ ]:
login.html:

In [ ]:
register.html:

### 9. Create a RESTful API using Flask to perform CRUD operations on resources like books or movies.

In [ ]:
from flask import Flask, jsonify, request

app = Flask(__name__)

# Sample initial data for books
books = [
    {'id': 1, 'title': 'Book 1', 'author': 'Author 1'},
    {'id': 2, 'title': 'Book 2', 'author': 'Author 2'},
    {'id': 3, 'title': 'Book 3', 'author': 'Author 3'}
]

# Route to get all books
@app.route('/books', methods=['GET'])
def get_books():
    return jsonify(books)

# Route to get a specific book by its ID
@app.route('/books/<int:book_id>', methods=['GET'])
def get_book(book_id):
    book = next((book for book in books if book['id'] == book_id), None)
    if book:
        return jsonify(book)
    return jsonify({'message': 'Book not found'}), 404

# Route to add a new book
@app.route('/books', methods=['POST'])
def add_book():
    new_book = {
        'id': len(books) + 1,
        'title': request.json['title'],
        'author': request.json['author']
    }
    books.append(new_book)
    return jsonify(new_book), 201

# Route to update a book by its ID
@app.route('/books/<int:book_id>', methods=['PUT'])
def update_book(book_id):
    book = next((book for book in books if book['id'] == book_id), None)
    if book:
        book['title'] = request.json.get('title', book['title'])
        book['author'] = request.json.get('author', book['author'])
        return jsonify(book)
    return jsonify({'message': 'Book not found'}), 404

# Route to delete a book by its ID
@app.route('/books/<int:book_id>', methods=['DELETE'])
def delete_book(book_id):
    global books
    books = [book for book in books if book['id'] != book_id]
    return jsonify({'message': 'Book deleted'})

if __name__ == '__main__':
    app.run(debug=True)


### 10. Design a Flask app with proper error handling for 404 and 500 errors.

In [ ]:
from flask import Flask, render_template

app = Flask(__name__)

# Custom error handler for 404 Not Found error
@app.errorhandler(404)
def not_found_error(error):
    return render_template('error.html', error_code=404, error_message="Page not found"), 404

# Custom error handler for 500 Internal Server Error
@app.errorhandler(500)
def internal_error(error):
    return render_template('error.html', error_code=500, error_message="Internal Server Error"), 500

# Route that intentionally raises a 500 error for testing purposes
@app.route('/raise_error')
def raise_error():
    # Simulate an internal server error by dividing by zero (which will raise an exception)
    result = 1 / 0
    return result

if __name__ == '__main__':
    app.run(debug=True)


In [ ]:
error.html:

In [ ]:
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>Error</title>
</head>
<body>
    <h1>Error {{ error_code }}</h1>
    <p>{{ error_message }}</p>
</body>
</html>

## Real-time Development:

### 11. Create a real-time chat application using Flask-SocketIO.

In [ ]:
from flask import Flask, render_template
from flask_socketio import SocketIO

app = Flask(__name__)
app.config['SECRET_KEY'] = 'secret'
socketio = SocketIO(app)

@app.route('/')
def index():
    return render_template('index.html')

@socketio.on('message')
def handle_message(message):
    print('Received message: ' + message)
    socketio.emit('message', message)  # Broadcast the message to all connected clients

if __name__ == '__main__':
    socketio.run(app, debug=True)


In [ ]:
Create an index.html file in a templates folder:
index.html:

In [ ]:
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>Chat Application</title>
    <script src="https://cdnjs.cloudflare.com/ajax/libs/socket.io/4.3.2/socket.io.js"></script>
    <script type="text/javascript" charset="utf-8">
        document.addEventListener('DOMContentLoaded', () => {
            var socket = io.connect('http://' + document.domain + ':' + location.port);
            socket.on('message', function(msg) {
                var ul = document.getElementById('messages');
                var li = document.createElement('li');
                li.appendChild(document.createTextNode(msg));
                ul.appendChild(li);
            });

            document.getElementById('send').onclick = function() {
                socket.send(document.getElementById('message').value);
                document.getElementById('message').value = '';
            };
        });
    </script>
</head>
<body>
    <ul id="messages"></ul>
    <input type="text" id="message">
    <button id="send">Send</button>
</body>
</html>

### 12. Build a Flask app that updates data in real-time using WebSocket connections.

In [ ]:
from flask import Flask, render_template
from flask_socketio import SocketIO, emit
import time
import threading

app = Flask(__name__)
app.config['SECRET_KEY'] = 'secret'
socketio = SocketIO(app)

# Simulated data that will be updated
data = {'value': 0}

# Function to simulate data updates
def update_data():
    global data
    while True:
        data['value'] += 1
        socketio.emit('update', data['value'])  # Broadcast updated data to all connected clients
        time.sleep(1)

@app.route('/')
def index():
    return render_template('index.html')

@socketio.on('connect')
def handle_connect():
    emit('update', data['value'])  # Send initial data when a client connects

if __name__ == '__main__':
    # Start the background thread to update data
    update_thread = threading.Thread(target=update_data)
    update_thread.daemon = True
    update_thread.start()

    socketio.run(app, debug=True)

In [ ]:
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>Real-Time Data Update</title>
    <script src="https://cdnjs.cloudflare.com/ajax/libs/socket.io/4.3.2/socket.io.js"></script>
    <script type="text/javascript" charset="utf-8">
        document.addEventListener('DOMContentLoaded', () => {
            var socket = io.connect('http://' + document.domain + ':' + location.port);
            socket.on('update', function(value) {
                document.getElementById('data').innerText = value;
            });
        });
    </script>
</head>
<body>
    <h1>Real-Time Data Update</h1>
    <p>Data Value: <span id="data"></span></p>
</body>
</html>


### 13. Implement notifications in a Flask app using websockets to notify users of updates.

In [ ]:
from flask import Flask, render_template
from flask_socketio import SocketIO, emit

app = Flask(__name__)
app.config['SECRET_KEY'] = 'secret'
socketio = SocketIO(app)

# Simulated notification updates
notifications = []

@app.route('/')
def index():
    return render_template('index.html')

@socketio.on('connect')
def handle_connect():
    emit('initial_notifications', notifications)  # Send initial notifications when a client connects

@app.route('/add_notification/<message>')
def add_notification(message):
    notifications.append(message)
    socketio.emit('new_notification', message, broadcast=True)  # Send new notification to all connected clients
    return f'Added notification: {message}'

if __name__ == '__main__':
    socketio.run(app, debug=True)


In [ ]:
HTML Template:

Create an index.html file in a templates folder:

index.html:

<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>Real-Time Notifications</title>
    <script src="https://cdnjs.cloudflare.com/ajax/libs/socket.io/4.3.2/socket.io.js"></script>
    <script type="text/javascript" charset="utf-8">
        document.addEventListener('DOMContentLoaded', () => {
            var socket = io.connect('http://' + document.domain + ':' + location.port);
            socket.on('initial_notifications', function(notifications) {
                for (var i = 0; i < notifications.length; i++) {
                    displayNotification(notifications[i]);
                }
            });
            socket.on('new_notification', function(message) {
                displayNotification(message);
            });

            function displayNotification(message) {
                var ul = document.getElementById('notifications');
                var li = document.createElement('li');
                li.appendChild(document.createTextNode(message));
                ul.appendChild(li);
            }
        });
    </script>
</head>
<body>
    <h1>Real-Time Notifications</h1>
    <ul id="notifications"></ul>
</body>
</html>
